In [2]:
import streamlit as st
import os

import urllib.request
import sys
import json

import re
from dotenv import load_dotenv
import time
from datetime import date

import streamlit as st
from google import genai
from google.genai import types

In [20]:
def get_todays_date():
    """
    Provides today's date

    Returns:
        provides today's date
    
    """

    return str(date.today())

In [21]:
def get_weather_at_location(location: str, date: str):
    api_location = re.sub(r'[^a-zA-Z0-9\s]', '', location).replace(" ", "%20")
    #print(api_location)
    load_dotenv(override=True)
    weather_key = st.secrets["WEATHER_KEY"]
    #print(weather_key)
    url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + api_location + "/" + date + "?unitGroup=us&include=hours&key=" + weather_key + "&contentType=json"
    #print(url)
    

    try: 
        ResultBytes = urllib.request.urlopen(url)
        # Parse the results as JSON
        jsonData = json.load(ResultBytes)
        
    except urllib.error.HTTPError  as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code, ErrorInfo)
        sys.exit()
    except  urllib.error.URLError as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code,ErrorInfo)
        sys.exit()
    return jsonData

In [22]:
def parse_weather_for_dewtemp(hour_of_day: int, forecast: dict):
    dew = forecast['days'][0]['hours'][hour_of_day]['dew']
    temp = forecast['days'][0]['hours'][hour_of_day]['temp']
    return temp, dew

In [28]:
def determine_adjustment(total):
  
    conversion_dict = {
        100: 0.0,
        110: 0.005,
        120: 0.01,
        130: 0.02,
        140: 0.03,
        150: 0.045,
        160: 0.06,
        170: 0.08,
        180: 0.1,
    }
    for key in conversion_dict.keys():
        if total > key and total <= key+10:
            lower = key
            upper = key+10
            lower_adjust = conversion_dict[key]
            upper_adjust = conversion_dict[key+10]
    range1_ratio = (total-lower)/(upper-lower)
    range2_equivalent = (upper_adjust-lower_adjust)*range1_ratio+lower_adjust
    return range2_equivalent

In [35]:
def adjust_target_pace(minutes: int, seconds: int, temp: float, dew: float):
    
    
    total_seconds = 60*minutes + seconds
    dewtemp = temp + dew


    #print(dewtemp_total)
    if dewtemp <= 100:
        return time.strftime("%M:%S", time.gmtime(total_seconds)), 0
    elif dewtemp >180:
        return "Not safe to run fast", "N/A"

    adjustment = determine_adjustment(dewtemp)
    #print(adjustment)
    adjusted_seconds = total_seconds*(adjustment+1)
    result = time.strftime("%M:%S", time.gmtime(adjusted_seconds))

    return result, adjustment

In [ ]:
def adjust_target_pace_from_weather_at_location(location: str, date: str, hour_of_day: int, pace_minutes: int, pace_seconds: int):
    """
    Adjusts a given target running pace based on the weather at the location on a certain date
    
    Args:
        location: the location that the runner will be running
        date: the date that the runner will be targeting the pace, in the format: YYYY-MM-DD
        hour_of_day: the hour of the day the runner will be running
        pace_minutes: the minute portion of the pace, ex: for 5:16 pace, this would be 5
        pace_seconds: the seconds portion of the pace, ex: for 5:16 pace, this would be 16

    Returns:
        the adjusted pace that the runner should target or "not safe to run fast" if the temperature + dew point is over 180
        the adjustment % used or "N/A" if the temperature + dew point is over 180
        the temperature for the date and time
        the dew point for the date and time
        
    
    """

    forecast = get_weather_at_location(location, date)

    temp, dew = parse_weather_for_dewtemp(hour_of_day,forecast)

    #get new pace and the adjustment %
    result, adjustment = adjust_target_pace(pace_minutes, pace_seconds, temp, dew)

    return result, adjustment, temp, dew

In [52]:
def adjust_performance_pace(minutes: int, seconds: int, temp: float, dew: float):
    
    
    total_seconds = 60*minutes + seconds
    dewtemp = temp + dew


    #print(dewtemp_total)
    if dewtemp <= 100:
        return time.strftime("%M:%S", time.gmtime(total_seconds)), 0

    adjustment = determine_adjustment(dewtemp)
    #print(adjustment)
    adjusted_seconds = total_seconds/(adjustment+1)
    result = time.strftime("%M:%S", time.gmtime(adjusted_seconds))

    return result, adjustment

In [ ]:
def adjust_performance_pace_from_weather_at_location(location: str, date: str, hour_of_day: int, pace_minutes: int, pace_seconds: int):
    """
    Adjusts a given performance running pace based on the weather at the location on a certain date
    
    Args:
        location: the location that the runner will be running
        date: the date that the runner achieved the performance pace, in the format: YYYY-MM-DD
        hour_of_day: the hour of the day the runner will be running
        pace_minutes: the minute portion of the pace, ex: for 5:16 pace, this would be 5
        pace_seconds: the seconds portion of the pace, ex: for 5:16 pace, this would be 16

    Returns:
        the adjusted pace for the performance the runner achieved
        the adjustment % used or "N/A" if the temperature + dew point is over 180
        the temperature for the date and time
        the dew point for the date and time
    """

    forecast = get_weather_at_location(location, date)

    temp, dew = parse_weather_for_dewtemp(hour_of_day,forecast)

    #get new pace and the adjustment %
    result, adjustment = adjust_performance_pace(pace_minutes, pace_seconds, temp, dew)

    return result, adjustment, temp, dew

In [50]:
date = "2025-06-28"
location = "Cottage Grove MN"
hour_of_day = 9
minutes = 7
seconds = 23

In [51]:
result, adjustment, temp, dew = adjust_performance_pace_from_weather_at_location(location, date, hour_of_day, minutes, seconds)
print(result, adjustment, temp, dew)


07:12 0.025 69.3 65.7


08:27 0.049200000000000015 81.4 71.4


In [47]:
date = "2025-06-28"
location = "Cottage Grove MN"
hours_of_day = 9
minutes = 7
seconds = 23
weather = get_weather_at_location(location, date)
temp, dew = parse_weather_for_dewtemp(hours_of_day, weather)
result, adjustment = adjust_performance_pace(minutes, seconds, temp, dew)


In [48]:
print(weather['days'][0]['hours'][hours_of_day])
print(temp, dew, dew+temp)
print(result, adjustment)


{'datetime': '09:00:00', 'datetimeEpoch': 1751119200, 'temp': 69.3, 'feelslike': 69.3, 'humidity': 88.22, 'dew': 65.7, 'precip': 0.0, 'precipprob': 0.0, 'snow': 0.0, 'snowdepth': 0.0, 'preciptype': None, 'windgust': 15.0, 'windspeed': 1.2, 'winddir': 45.0, 'pressure': 1013.1, 'visibility': 9.9, 'cloudcover': 3.4, 'solarradiation': 330.0, 'solarenergy': 1.2, 'uvindex': 3.0, 'severerisk': 5.0, 'conditions': 'Clear', 'icon': 'clear-day', 'stations': ['99999900180', 'KSGS', 'KSTP', '72658414927', '72660304974', 'KMSP', 'D3800'], 'source': 'obs'}
69.3 65.7 135.0
07:12 0.025
